In [1]:
import math
import cv2 as cv
import numpy as np
import pandas as pd
from PIL import Image
from glob import glob
from matplotlib import pyplot as plt
from matplotlib.patches import Ellipse

In [2]:
def img_precessing(img, medianblur_kernel=21, canny_threshold1=5, canny_threshold2=40):
    img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)                     # 轉成灰階
    img_blur = cv.medianBlur(img_gray, medianblur_kernel)              # median blur 
    # img_blur = img_gray                                                # no median blur
    img_edges= cv.Canny(img_blur, canny_threshold1, canny_threshold2)  # canny

    return img_gray, img_blur, img_edges

In [3]:
def Get_canny_angle(img, medianblur_kernel=21, canny_threshold1=5, canny_threshold2=40):
    canny_pred_angle=[]
    img_gray, blur, edges = img_precessing(img, medianblur_kernel, canny_threshold1, canny_threshold2)
    ###############
    cnt, hierarchy = cv.findContours(edges, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    '''找全部的輪廓出來畫'''
    allcnt=[]
    for c in cnt:
        allcnt.extend(c.tolist())
    allcnt = np.array(allcnt)

    '''有可能找不到輪廓!!'''
    if len(allcnt) != 0:
        ''' 找出橢圓 '''
        img_elli = img.copy()
        ellipse  = cv.fitEllipseAMS(allcnt)  # fit出橢圓
        # cv.ellipse(img_elli, ellipse, (0,0,255), 3)
        
        ####### 找出角度(11點鐘方向)
        img_11=img.copy()
        (xc, yc), (width, height), angle = ellipse     # (xc,yc):橢圓中心點 (width, height):橢圓寬長
        print("最原始angle= ", int(angle))
        
        if angle>90:  # 一開始的angle是比較長短邊的角度
            normal_angle = 360-angle-60
            print('normal angle=', normal_angle)
            
            ######################################## 處理特殊角度(150~180) ###############################################
            if normal_angle > 150 and normal_angle <=180:   
                normal_angle = normal_angle - 30
                canny_pred_angle.append(normal_angle)
                rmajor = height / 2
                xtop0  = xc + math.cos(math.radians(angle+120)) * rmajor
                ytop0  = yc + math.sin(math.radians(angle+120)) * rmajor
                rmajor = height / 2
                xtop11 = xc + math.cos(math.radians(angle+90)) * rmajor
                ytop11 = yc + math.sin(math.radians(angle+90)) * rmajor
                rminor = width / 2
                xtop = xc + math.cos(math.radians(angle+30)) * rminor
                ytop = yc + math.sin(math.radians(angle+30)) * rminor
                ellipse_info = Ellipse(xy=(xc, yc), width=width, height=height, angle=angle+30, edgecolor='b', fc='None', lw=2, linestyle='dotted')
            ######################################## 處理特殊角度(180~) ################################################
            elif normal_angle > 180:                         
                normal_angle = normal_angle - 60
                canny_pred_angle.append(normal_angle)
                rmajor = height / 2
                xtop0  = xc + math.cos(math.radians(angle+150)) * rmajor
                ytop0  = yc + math.sin(math.radians(angle+150)) * rmajor
                rmajor = height / 2
                xtop11 = xc + math.cos(math.radians(angle+120)) * rmajor
                ytop11 = yc + math.sin(math.radians(angle+120)) * rmajor
                rminor = width / 2
                xtop = xc + math.cos(math.radians(angle+60)) * rminor
                ytop = yc + math.sin(math.radians(angle+60)) * rminor
                ellipse_info = Ellipse(xy=(xc, yc), width=width, height=height, angle=angle+60, edgecolor='b', fc='None', lw=2, linestyle='dotted')
            ######################################## 正常角度(0~150) ################################################
            else:
                canny_pred_angle.append(normal_angle)
                rmajor = height / 2
                xtop0  = xc + math.cos(math.radians(angle+90)) * rmajor
                ytop0  = yc + math.sin(math.radians(angle+90)) * rmajor
                rmajor = height / 2
                xtop11 = xc + math.cos(math.radians(angle+60)) * rmajor
                ytop11 = yc + math.sin(math.radians(angle+60)) * rmajor
                rminor = width / 2
                xtop = xc + math.cos(math.radians(angle)) * rminor
                ytop = yc + math.sin(math.radians(angle)) * rminor
                ellipse_info = Ellipse(xy=(xc, yc), width=width, height=height, angle=angle, edgecolor='b', fc='None', lw=2, linestyle='dotted')

            print('new normal angle=', normal_angle)
            cv.line(img_11, (int(xtop0), int(ytop0)), (int(xc), int(yc)), (0, 0, 255), 1)
            cv.line(img_11, (int(xtop11), int(ytop11)), (int(xc), int(yc)), (128, 0, 0), 3)
            cv.line(img_11, (int(xtop), int(ytop)), (int(xc), int(yc)), (0, 0, 255), 1)
            


        if angle<90:
            normal_angle = 180-angle-60
            print('normal angle=', normal_angle)
            

            ######################################## 處理特殊角度(0~30) ###############################################  
            if normal_angle > 0 and normal_angle <= 30:   
                normal_angle = normal_angle + 60                            ## !!!!!!!!!!!!!
                canny_pred_angle.append(normal_angle)
                rmajor = height / 2
                xtop0  = xc + math.cos(math.radians(angle+210)) * rmajor
                ytop0  = yc + math.sin(math.radians(angle+210)) * rmajor
                rmajor = height / 2
                xtop11 = xc + math.cos(math.radians(angle+180)) * rmajor
                ytop11 = yc + math.sin(math.radians(angle+180)) * rmajor
                rminor = width / 2
                xtop = xc + math.cos(math.radians(angle+120)) * rminor
                ytop = yc + math.sin(math.radians(angle+120)) * rminor
                ellipse_info = Ellipse(xy=(xc, yc), width=width, height=height, angle=angle+120, edgecolor='b', fc='None', lw=2, linestyle='dotted')
            ######################################## 處理特殊角度(30~60) ###############################################
            elif normal_angle > 30 and normal_angle <= 60:   
                normal_angle = normal_angle + 30                            ## !!!!!!!!!!!!!!
                canny_pred_angle.append(normal_angle)
                rmajor = height / 2
                xtop0  = xc + math.cos(math.radians(angle+240)) * rmajor
                ytop0  = yc + math.sin(math.radians(angle+240)) * rmajor
                rmajor = height / 2
                xtop11 = xc + math.cos(math.radians(angle+210)) * rmajor
                ytop11 = yc + math.sin(math.radians(angle+210)) * rmajor
                rminor = width / 2
                xtop = xc + math.cos(math.radians(angle+150)) * rminor
                ytop = yc + math.sin(math.radians(angle+150)) * rminor
                ellipse_info = Ellipse(xy=(xc, yc), width=width, height=height, angle=angle+150, edgecolor='b', fc='None', lw=2, linestyle='dotted')
            ######################################## 正常角度(0~150) ################################################
            else:
                canny_pred_angle.append(normal_angle)
                rmajor = height / 2
                xtop0  = xc + math.cos(math.radians(angle+270)) * rmajor
                ytop0  = yc + math.sin(math.radians(angle+270)) * rmajor
                rmajor = height / 2
                xtop11 = xc + math.cos(math.radians(angle+240)) * rmajor
                ytop11 = yc + math.sin(math.radians(angle+240)) * rmajor
                rminor = width / 2
                xtop = xc + math.cos(math.radians(angle+180)) * rminor
                ytop = yc + math.sin(math.radians(angle+180)) * rminor
                ellipse_info = Ellipse(xy=(xc, yc), width=width, height=height, angle=angle+180, edgecolor='b', fc='None', lw=2, linestyle='dotted')
            
            print('new normal angle=', normal_angle)
            cv.line(img_11, (int(xtop0), int(ytop0)), (int(xc), int(yc)), (0, 0, 255), 1)
            cv.line(img_11, (int(xtop11), int(ytop11)), (int(xc), int(yc)), (128, 0, 0), 3)
            cv.line(img_11, (int(xtop), int(ytop)), (int(xc), int(yc)), (0, 0, 255), 1)

        
        ################# plot img
        # plt.figure()
        # f, axarr = plt.subplots(1, 5,figsize=(20,8)) 
        # axarr[0].imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))
        # axarr[1].imshow(cv.cvtColor(img_gray, cv.COLOR_BGR2RGB))
        # axarr[2].imshow(cv.cvtColor(blur, cv.COLOR_BGR2RGB))
        # axarr[3].imshow(cv.cvtColor(edges, cv.COLOR_BGR2RGB))
        # axarr[4].imshow(cv.cvtColor(img_11, cv.COLOR_BGR2RGB))
        # plt.show()
        ##################### save img
        # plt.imsave("../dataset/training_dataset/img/train/Crop_Gray/{}".format(img_name), cv.cvtColor(img_gray, cv.COLOR_BGR2RGB))       # !!!!!!!!!!!!
        # plt.imsave("../dataset/training_dataset/img/train/Crop_MedianBlur/{}".format(img_name), cv.cvtColor(blur, cv.COLOR_BGR2RGB))     # !!!!!!!!!!!!
        # plt.imsave("../dataset/training_dataset/img/train/Crop_CannyEdge/{}".format(img_name), cv.cvtColor(edges, cv.COLOR_BGR2RGB))     # !!!!!!!!!!!!
        # plt.imsave("../dataset/result_img/canny_traindata_direction_result/canny_{}".format(img_name), cv.cvtColor(img_11, cv.COLOR_BGR2RGB))  # !!!!!!!!!!!!

        # plt.figure()
        # f, ax = plt.subplots(figsize=(3,4)) 
        # ax.add_artist(ellipse_info)  # 橢圓
        # ax.set_axis_off()
        # plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))
        # plt.savefig("../dataset/result_img/canny_traindata_ellipse_result/ellipse_{}".format(img_name),bbox_inches='tight', pad_inches=0)    # !!!!!!!!!!!!
        # plt.close(f)

    return canny_pred_angle

In [4]:
# 測試集真值dataframe
tes_df = pd.read_csv('../dataset/test_dataset/direc/old_angle_20221222.csv') 
# tes_df = pd.read_csv('../dataset/training_dataset/direc/train/old_angle_20221222.csv')
# tes_df = pd.read_csv('../dataset/training_dataset/direc/validation/old_angle_20221222.csv')

# 測試集image path
tes_img_path = "../dataset/test_dataset/img/unCrop/"
# tes_img_path = "../dataset/training_dataset/img/train/unCrop/"
# tes_img_path = "../dataset/training_dataset/img/validation/unCrop/"

# stage1 & stage2 結果
stage1Andstage2_result_df = pd.read_csv('../../yolov4_keras/result_df/test_stage1Andstage2_result.csv')
# stage1Andstage2_result_df = pd.read_csv('../../yolov4_keras/result_df/test_stage1Andstage2_result_nosmooth.csv')      # 
# stage1Andstage2_result_df = pd.read_csv('../../yolov4_keras/result_df/train_stage1Andstage2_result.csv')
# stage1Andstage2_result_df = pd.read_csv('../../yolov4_keras/result_df/validation_stage1Andstage2_result.csv')
stage1Andstage2_result_df.head(8)

,test_name,test_true,stage1_test_pred,true_bbox,pred_bbox,score
0,h_000058.png,0,1,NO,"(1, 1, 1, 1)",0.000000
1,h_000058_copy.png,0,1,NO,"(1, 1, 1, 1)",0.000000
2,l_000019.png,0,1,NO,"(1, 1, 1, 1)",0.000000
3,q_000027.png,0,1,NO,"(1, 1, 1, 1)",0.000000
4,q_000027_copy.png,0,1,NO,"(1, 1, 1, 1)",0.000000
5,q_000031.png,0,1,NO,"(1, 1, 1, 1)",0.000000
6,h_000048.png,1,1,"(432, 24, 659, 281)","(358, 60, 731, 564)",0.494053
7,h_000049.png,1,1,"(273, 19, 529, 297)","(163, 0, 724, 632)",0.479866


In [5]:
tes_df.head(2)

,img_name,x1,y1,x2,y2,bbox,old_angle
0,h_000048.png,453,27,477,117,"(432, 24, 659, 281)",105.0
1,h_000049.png,303,23,323,109,"(273, 19, 529, 297)",103.0


# Old
# img -> stage 3
img 不切割直接進去做影像處理 

In [6]:
row, col = tes_df.shape
print(" [perfect stage 1 & stage 2] = (row, col)",(row,col))

 [perfect stage 1 & stage 2] = (row, col) (189, 7)


In [7]:
angle_errors = 0
perfect_TP = 0
for r in range(row):         # row: 列數
    test_img_name = tes_df.iloc[r:r+1]['img_name'][r]
    test_true     = tes_df.iloc[r:r+1]['old_angle'][r]
    # pred_bbox     = tes_df.iloc[r:r+1]['bbox'][r]
    
    '''perfect TP'''
    perfect_TP += 1
    '''crop image by bbox''' 
    # bbox = [int(i.replace("(","").replace(")","").replace(" ","")) for i in pred_bbox.split(",")]
    img  = Image.open(tes_img_path+test_img_name)  # read image
    # img  = img.crop(bbox)
    img  = np.array(img)
        
    '''預測值&真值'''
    canny_pred_angle = Get_canny_angle(img, medianblur_kernel=9, canny_threshold1=5, canny_threshold2=40)[0]
    true_angle       = tes_df[tes_df['img_name']==test_img_name]['old_angle'].values[0]
    angle_errors += abs(canny_pred_angle-true_angle)

最原始angle=  96
normal angle= 203.53904724121094
new normal angle= 143.53904724121094
最原始angle=  118
normal angle= 181.07610321044922
new normal angle= 121.07610321044922
最原始angle=  92
normal angle= 207.4381866455078
new normal angle= 147.4381866455078
最原始angle=  173
normal angle= 126.41928100585938
new normal angle= 126.41928100585938
最原始angle=  5
normal angle= 114.96866607666016
new normal angle= 114.96866607666016
最原始angle=  6
normal angle= 113.25597286224365
new normal angle= 113.25597286224365
最原始angle=  4
normal angle= 115.51323318481445
new normal angle= 115.51323318481445
最原始angle=  161
normal angle= 138.6610870361328
new normal angle= 138.6610870361328
最原始angle=  151
normal angle= 148.20562744140625
new normal angle= 148.20562744140625
最原始angle=  149
normal angle= 150.11502075195312
new normal angle= 120.11502075195312
最原始angle=  17
normal angle= 102.9879035949707
new normal angle= 102.9879035949707
最原始angle=  160
normal angle= 139.7266387939453
new normal angle= 139.72663879394

In [8]:
#perfect_TP #189
perfect_TN   = 181
angle_errors = angle_errors + perfect_TN*180   # 如果直接進stage3，沒有壺腹乳頭影像也會預測角度，所以要給180度懲罰

perfect_mae  = angle_errors/(perfect_TP+perfect_TN)
perfect_pmae = perfect_mae/180 * 100
print('mae={}, pmae={}%'.format(round(perfect_mae,2), round(perfect_pmae, 2)))

mae=100.69, pmae=55.94%


# Ideal
# [perfect stage 1 & stage 2] & stage 3

In [9]:
row, col = tes_df.shape
print(" [perfect stage 1 & stage 2] = (row, col)",(row,col))

 [perfect stage 1 & stage 2] = (row, col) (189, 7)


In [10]:
angle_errors = 0
perfect_TP = 0
for r in range(row):         # row: 列數
    test_img_name = tes_df.iloc[r:r+1]['img_name'][r]
    test_true     = tes_df.iloc[r:r+1]['old_angle'][r]
    pred_bbox     = tes_df.iloc[r:r+1]['bbox'][r]
    
    '''perfect TP'''
    perfect_TP += 1
    '''crop image by bbox''' 
    bbox = [int(i.replace("(","").replace(")","").replace(" ","")) for i in pred_bbox.split(",")]
    img  = Image.open(tes_img_path+test_img_name)  # read image
    img  = img.crop(bbox)
    img  = np.array(img)
        
    '''預測值&真值'''
    canny_pred_angle = Get_canny_angle(img, medianblur_kernel=9, canny_threshold1=5, canny_threshold2=40)[0]
    true_angle       = tes_df[tes_df['img_name']==test_img_name]['old_angle'].values[0]
    angle_errors += abs(canny_pred_angle-true_angle)

最原始angle=  6
normal angle= 113.95275211334229
new normal angle= 113.95275211334229
最原始angle=  5
normal angle= 114.85236930847168
new normal angle= 114.85236930847168
最原始angle=  21
normal angle= 98.19572448730469
new normal angle= 98.19572448730469
最原始angle=  15
normal angle= 104.76766967773438
new normal angle= 104.76766967773438
最原始angle=  15
normal angle= 104.40846252441406
new normal angle= 104.40846252441406
最原始angle=  4
normal angle= 115.12610578536987
new normal angle= 115.12610578536987
最原始angle=  5
normal angle= 114.26096773147583
new normal angle= 114.26096773147583
最原始angle=  0
normal angle= 119.06981509923935
new normal angle= 119.06981509923935
最原始angle=  56
normal angle= 63.23189163208008
new normal angle= 63.23189163208008
最原始angle=  17
normal angle= 102.84802055358887
new normal angle= 102.84802055358887
最原始angle=  9
normal angle= 110.40223979949951
new normal angle= 110.40223979949951
最原始angle=  11
normal angle= 108.41536617279053
new normal angle= 108.41536617279053
最原

In [11]:
perfect_TN = 181
perfect_mae  = angle_errors/(perfect_TP+perfect_TN)
perfect_pmae = perfect_mae/180 * 100
print('mae={}, pmae={}%'.format(round(perfect_mae,2), round(perfect_pmae, 2)))

mae=8.19, pmae=4.55%


# Practical
# stage 1 & stage 2 & stage 3   

In [12]:
row, col = stage1Andstage2_result_df.shape
print("stage1Andstage2_result_df = (row, col)",(row,col))

stage1Andstage2_result_df = (row, col) (195, 6)


In [13]:
medianblur_kernel=9
canny_threshold1 =5
canny_threshold2 =40

# varify
TpAndFp=0
Fn     =0
Fp     =0

angle_errors = 0

for r in range(row):         # row: 列數
    test_img_name    = stage1Andstage2_result_df.iloc[r:r+1]['test_name'][r]
    test_true        = stage1Andstage2_result_df.iloc[r:r+1]['test_true'][r]
    stage1_test_pred = stage1Andstage2_result_df.iloc[r:r+1]['stage1_test_pred'][r]
    pred_bbox        = stage1Andstage2_result_df.iloc[r:r+1]['pred_bbox'][r]
    
    
    '''TP&FP'''
    if test_true==1 and stage1_test_pred==1:
        TpAndFp+=1
        '''crop image by bbox''' 
        bbox = [int(i.replace("(","").replace(")","").replace(" ","")) for i in pred_bbox.split(",")]
        img  = Image.open(tes_img_path+test_img_name)  # read image
        img  = img.crop(bbox)
        img  = np.array(img)
        
        '''預測值&真值'''
        try:
            canny_pred_angle = Get_canny_angle(img, medianblur_kernel, canny_threshold1, canny_threshold2)[0]
            true_angle       = tes_df[tes_df['img_name']==test_img_name]['old_angle'].values[0]
            angle_errors += abs(canny_pred_angle-true_angle)
        except:
            print("===============================",test_img_name)
        
    # '''FN'''
    if test_true==1 and stage1_test_pred==0:
        Fn+=1
        angle_errors+=180

    # '''FP'''
    if test_true==0 and stage1_test_pred==1:
        Fp+=1
        angle_errors+=180

最原始angle=  3
normal angle= 116.0997474193573
new normal angle= 116.0997474193573
最原始angle=  9
normal angle= 110.57741165161133
new normal angle= 110.57741165161133
最原始angle=  5
normal angle= 114.94575023651123
new normal angle= 114.94575023651123
最原始angle=  1
normal angle= 118.41712427139282
new normal angle= 118.41712427139282
最原始angle=  7
normal angle= 112.45634412765503
new normal angle= 112.45634412765503
最原始angle=  9
normal angle= 110.14664936065674
new normal angle= 110.14664936065674
最原始angle=  1
normal angle= 118.92460656166077
new normal angle= 118.92460656166077
最原始angle=  7
normal angle= 112.43452596664429
new normal angle= 112.43452596664429
最原始angle=  175
normal angle= 124.55009460449219
new normal angle= 124.55009460449219
最原始angle=  176
normal angle= 123.61109924316406
new normal angle= 123.61109924316406
最原始angle=  5
normal angle= 114.90649127960205
new normal angle= 114.90649127960205
最原始angle=  4
normal angle= 115.41080665588379
new normal angle= 115.41080665588379
最原

In [14]:
All = 370
Tn = All-TpAndFp-Fn-Fp

print("(Tp+Fp, Fn, Fp, Tn)",(TpAndFp, Fn, Fp, Tn))

(Tp+Fp, Fn, Fp, Tn) (166, 23, 6, 175)


In [15]:
mae  = angle_errors/(TpAndFp+Fn+Fp+Tn)
pmae = mae/180 * 100
print('mae={}, pmae={}%'.format(round(mae,2), round(pmae, 2)))

mae=22.77, pmae=12.65%
